# [이론5] 문맥을 반영한 워드 임베딩 : BERT (Contextual Word Embedding: BERT)

---

## 학습 목표
- Transformer에 대해 살펴봅시다.
- BERT에 대해 살펴봅시다.

---

## 목차

### [이론5] 문맥을 반영한 워드 임베딩 : BERT (Contextual Word Embedding: BERT)
1. 소개
2. Transformer
3. BERT

---

##  [이론5] 문맥을 반영한 워드 임베딩 : BERT (Contextual Word Embedding: BERT)

### 1. 소개

이전 시간에 우리는 단어 표현에 대해 공부하였습니다.

그 때 단어 표현으로 크게 네 가지가 있다고 하였습니다.

- 개수 기반(Count-based)
  - 근처 단어들의 개수를 새어서 만드는 방법
  - 예: tf-idf, PPMI
- 클래스 기반(Class-based)
  - 계층적 클러스터링을 통해 만드는 방법
  - 예: Brown clusters
- 분산 추론 기반 임베딩(Distributed prediction-based embeddings)
  - 근처 단어 및 멀리 있는 단어들을 구분하는 분류기(classifier)를 학습시켜 만드는 방법
  - 예: Word2vec, Fasttext
- 분산 문맥 임베딩(Distributed contextual embeddings)
  - 언어 모델(language model)에서 나온 임베딩
  - 예: ELMo, BERT, GPT

이번 시간에는 마지막에 있는 '분산 문맥 임베딩'의 하나인 BERT에 대해 살펴보겠습니다.

지난 이론 시간에 배운 '분산 추론 기반 임베딩'인 Word2vec이나 fastText는 여러 한계점이 있습니다.

그 중 하나로, 단어는 단 하나의 뜻만을 가지고 있지 않고 단어의 뜻을 추론하기 위해서는 문맥을 고려해야 합니다.
예를 들어 아래 문장들을 보겠습니다.

- Elmo and Cookie Monster **play** a game.
- The Broadway **play** premiered yesterday.
- Flowers **play** an important role in mental health of people.
- Girls will **play** characters with a role to build the group’s social relationship.
- Some **play** the piano, while others dance, sing, and perform plays.

이 문장들은 모두 `play`라는 단어를 가지고 있습니다.

하지만 해당 단어는 여러가지 뜻을 가지고 있습니다.

<img src="img/play.jpg" width="50%" height="50%" title="play" alt="play"></img>

출처: https://dic.daum.net/word/view.do?wordid=ekw000127629

이처럼 다양한 뜻을 가지는 단어를 단 하나의 벡터로 표현하는 것은 한계점이 있습니다.

그리고 그 단어의 뜻에 대한 이해를 위해서는 해당 단어가 어느 문맥에서 쓰였는지 알아야 합니다.

즉, 단어와 함께 쓰여지는 **옆에 존재하는 단어들과 함께 살펴봐야지** 그 단어를 제대로 이해할 수 있는 것입니다.

그럼 이러한 단어 그리고 문맥에 대한 이해를 위해서는 어떤 모델을 사용해야할까요?

여러가지가 있을 수 있지만 그 중 가장 적합한 것은 이전 시간에 배운 언어 모델입니다.

왜냐하면 언어 모델이란 어떤 문장 혹은 연속된 단어 조합이 주어졌을 때 그 단어가 생성될 확률을 계산하기 때문입니다.

$P(W) = P(w_1 w_2 \ldots w_k)$

예를 들어 좋은 언어 모델이 있을 때 아래 두 문장의 확률을 다음과 같이 나타낼 것입니다.

- P(나는 인공지능 자연어처리 공부가 좋다) = 0.7
- P(나는 좋다 자연어처리 공부가 인공지능) = 0.2

그리고 이런 언어 모델을 통해 다음 단어를 예측하는 것도 가능합니다.

<img src="img/keywordexpansion.png" width="50%" height="50%" title="keywordexpansion" alt="keywordexpansion"></img>

이처럼 주어진 글의 문맥을 이해함에 있어 언어 모델은 도움이 됩니다.

이런 언어 모델을 n-gram 모델이 아닌 신경망을 통해 만들어보는 아이디어로 시작할 수 있습니다.

### 2. Transformer

신경망을 이용한 언어 모델의 가장 간단한 것이 바로 RNN입니다.

왜냐하면 주어진 단어들을 순서대로 받아 단어 간의 순서를 이해하고 그 관계를 학습하는 것이 바로 RNN이기 때문입니다.

<img src="img/RNN_2.png" width="50%" height="50%" title="RNN_2" alt="RNN_2"></img>

출처: https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks

하지만 RNN의 가장 큰 단점은 학습 속도가 느리다는 점이 있습니다.

왜냐하면 시계열 데이터를 하나씩 처리하는 하다보니 데이터 간의 종속성으로 인해 병렬 계산을 할 수 없기 때문입니다.

이런 단점은 최근에 수많은 데이터를 빠르게 학습시켜야 하는 요구 사항에 부합하지 못하다는 단점이 있습니다.

이를 해결하기 위해 등장한 것이 Attention을 활용한 Transformer입니다.

#### Attention

Transformer 설명에 앞서 여기에 핵심적으로 쓰이는 Attention에 대해 먼저 살펴보겠습니다.

사람은 정보를 처리할 때, 보이는 모든 것을 다 살펴보는 것이 아니라 특정한 파트를 집중해서 봅니다.

<img src="img/attention_video.png" width="75%" height="75%" title="attention_video" alt="attention_video"></img>

출처: https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html

위 사진에서 여성(woman)을 찾을 때는 사진 전체를 보는 것이 아니라 실제 여성의 파트를 집중해서 봅니다.

원판(frisbee)를 찾을 때는 또 그것이 있는 파트를 집중해서 보게 되는 것이지요.

이를 통해 전체를 한꺼번에 이해하는 것보다 좀 더 효율적으로 내용을 파악할 수 있습니다.

이러한 집중은 텍스트 이해에서도 나타납니다.

즉, 하나의 파트를 이해함에 있어 다른 파트들을 전부 다 보는 것이 아니라 그와 관련이 높은 것에 집중해서 봅니다.

<img src="img/sentence-example-attention.png" width="50%" height="50%" title="sentence-example-attention" alt="sentence-example-attention"></img>

출처: https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html

위의 예시처럼 eating 단어를 이해함에 있어 중요한 단어는 apple입니다. 사과를 먹었다는 것이 중요하지요.

하지만 green이라는 단어는 그리 중요하지 않습니다. 먹는다는 행동에 있어 녹색은 큰 상관이 없으니까요.

이처럼 문맥 속 단어를 이해함에 있어 단어 간 집중함이 중요함을 알 수 있습니다.

#### Self-attention

이러한 Attention은 RNN의 출력과 결합하여 사용되었습니다.

여기서 발전하여 자기 자신의 문맥을 이해함에 있어 그 스스로 attention을 구하는 방법이 제안되었습니다.

그것이 self-attention입니다.

<img src="img/transformer_self-attention_visualization.png" width="50%" height="50%" title="transformer_self-attention_visualization" alt="transformer_self-attention_visualization"></img>

출처: http://jalammar.github.io/illustrated-transformer/

이를 통해 앞서 얘기드린대로 각 단어와 연관성이 높은 다른 단어들을 찾아낼 수 있습니다.

위의 그림에서 보듯 'it'이라는 단어가 'animal'과 'tired' 단어들과 연관성이 높은 즉, attention 값이 높음을 스스로 학습할 수 있습니다.

<img src="img/transformer_self_attention_vectors.png" width="50%" height="50%" title="transformer_self_attention_vectors" alt="transformer_self_attention_vectors"></img>

출처: http://jalammar.github.io/illustrated-transformer/

이를 위해서 Transformer에서는 세 가지 중요한 변수가 있습니다.

- Query: 얻고자 하는 context 벡터
- Key: Query와의 상관관계를 나타내고자 하는 대상
- Value: Query와 Key의 상관관계 값

문장 속 단어들을 위의 세 종류의 벡터로 표현합니다.

그 후 하나의 query에 대해 다른 단어 key들(자기 자신도 포함) 벡터를 내적하여 softmax를 구해 이를 value와 결합시켜 해당 query 단어가 문장 속에서 가지는 문맥/의미를 표현하도록 합니다.

이를 문장 속 모든 단어에 대해서 수행함으로써 전체적으로 문맥을 표현하는 벡터를 얻을 수 있습니다.

이것이 바로 Transformer입니다.

### 3. BERT

이제 Transformer를 이용하여 만든 BERT에 대해 살펴보겠습니다.

#### Byte Pair Encoding

이전에 fastText를 배우면서 subword를 표현하는 방법에 대해 배웠습니다.

그 때는 n-gram을 이용하였는데 이번에는 다른 방법인 Byte Pair Encoding(BPE)에 대해 살펴보겠습니다.

BPE는 원래 압축 알고리즘입니다.

다음과 같은 텍스트가 있다고 가정하겠습니다.

`aaabdaaabac`

BPE는 이 중 가장 많이 등장하는 단어 쌍(pair)를 찾습니다.

우리는 여기서 `aa`가 가장 많이 나타났음을 알 수 있습니다. 따라서 `aa`를 `Z`라는 단어로 바꿔보겠습니다.

`ZabdZabac`

길이가 줄어든 것을 알 수 있습니다.

그 다음 역시 찾아보면 `ab` 단어가 많이 나타납니다. 따라서 `ab`를 `Y`라는 단어로 바꿔보겠습니다.

`ZYdZYac`

이제 `ZY`가 많이 나타나는 것을 알 수 있습니다. 따라서 `ZY`를 `X`라는 단어로 바꿔보겠습니다.

`XdXac`

원래 길이 11의 텍스트가 5로 줄어든 것을 알 수 있습니다. 그 대신 dictionary를 가지게 됩니다.

- `aa` => `Z`
- `ab` => `Y`
- `ZY` => `X`

이 dictionary를 통해 우리는 `XdXac`라는 압축된 텍스트를 원래 텍스트인 `aaabdaaabac`로 복원할 수 있습니다.

이러한 BPE를 subword 표현에 어떻게 사용하는 것일까요?

BPE의 핵심 아이디어는 자주 나타나는 쌍(pair)을 묶자는 것입니다.

이는 NLP에서 어떤 의미일까요?

데이터에서 자주 나타나는 문자쌍은 어떤 단어를 만드는데 자주 사용된다는 것입니다.

그러한 문자쌍을 기억하고 있다면 처음 보는 단어(out of vocabulary)가 나타났을 때 어떤 문자쌍을 이용하였는지에 맞춰 단어를 추론할 수 있습니다. 이는 fastText처럼 모든 n-gram을 알아야 하는 것보다 효율적이라는 것을 알 수 있습니다.

예시와 함께 살펴보겠습니다.

주어진 텍스트 데이터가 있습니다.

거기서 우리는 단어들을 token으로 하여 만듭니다. 그러면서 동시에 해당 단어들이 몇 번 데이터에서 나타났는지를 기록합니다.

그렇게 나타난 단어와 그 횟수는 다음과 같습니다.

- low: 5번
- lower: 2번
- newest: 6번
- widest: 3번

이제 단어를 문자 단위로 쪼개어보겠습니다.

- l o w : 5번
- l o w e r : 2번
- n e w e s t : 6번
- w i d e s t : 3번

그리고 BPE를 사용하겠습니다.

BPE에 따라 많이 나타나는 문자 쌍을 찾으면 그것은 `es`입니다. 총 9번 등장하죠. (newest에서 6번, widest에서 3번)

그럼 `es`를 묶어 하나의 subword로 등록합니다.

- l o w : 5번
- l o w e r : 2번
- n e w es t : 6번
- w i d es t : 3번

다음으로는 `est`가 9번 나타납니다. 따라서 `est`를 묶어 하나의 subword로 등록합니다.

- l o w : 5번
- l o w e r : 2번
- n e w est : 6번
- w i d est : 3번

이 작업을 계속해서 반복해나갑니다. 그럼 아래와 같은 subword 토큰들을 만들 수 있습니다.

- l
- o
- w
- e
- r
- n
- w
- s
- t
- i
- d
- es
- est
- lo
- low
- ne
- new
- newest
- wi
- wid
- widest

만약 `lowest`라는 새로운 단어가 나타났다고 합시다.

그렇다면 우리가 가지고 있는 토큰들을 조합해보면 `low`와 `est`로 해당 단어를 표현할 수 있습니다.

이를 통해 그 단어가 어떤 뜻을 가지는지를 유추할 수 있습니다.

BERT에서는 이러한 BPE 방법을 통해 subword token을 만들어 사용합니다.

#### Training

<img src="img/BERT-language-modeling-masked-lm.png" width="50%" height="50%" title="BERT-language-modeling-masked-lm" alt="BERT-language-modeling-masked-lm"></img>

출처: http://jalammar.github.io/illustrated-bert/

BERT를 학습하는 방법은 임의로 숨긴 것을 복원시키는 방법 (Masked Language Model, MLM) 입니다.

주어진 텍스트가 있을 때 그 속에 있는 단어를 임의로 삭제(mask)합니다.

그 후 BERT는 해당 단어가 무엇인지 다른 단어들로부터 유추하여 맞춰야합니다.

학습에 있어 새로운 데이터를 만들어내는 것이 아니라, 기존의 데이터를 활용해서 학습을 진행합니다.

이러한 학습 방법을 self-supervised learning이라고 불립니다.

#### Benefits

<img src="img/bert_application.jpg" width="50%" height="50%" title="bert_application" alt="bert_application"></img>

출처: https://arxiv.org/abs/1810.04805

BERT가 당시 주목을 받았던 이유는 여러가지가 있지만, 그 중 가장 큰 장점은 다음과 같습니다.

NLP의 여러 application에 BERT를 이용하면 성능이 당시 최고 성능(State of the Art, SOTA)에 근접하거나 혹은 뛰어넘었기 때문입니다.

그 점이 NLP를 하는 여러 사람들에게 크게 매력적으로 다가왔고 그렇기에 인기가 많은 것입니다.

<img src="img/PLMfamily.jpg" width="75%" height="75%" title="PLMfamily" alt="PLMfamily"></img>

출처: https://github.com/thunlp/PLMpapers

BERT 외에 다양한 언어 모델이 많습니다.

왜냐하면 앞서 설명한 대로, 좋은 언어 모델이 있으면 그것을 통해 좋은 성능을 가지는 다양한 NLP application을 만들어 낼 수 있기 때문입니다.

위의 그림은 그 일부를 보여준 것으로 여러 연구자들이 좋은 언어 모델을 만들어내고 있습니다.

---

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>